In [1]:
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.interpolate import griddata
import scipy.io
import tensorflow as tf

Using backend: tensorflow.compat.v1



Instructions for updating:
non-resource variables are not supported in the long term



In [ ]:
def PDE(x,y):
    
    # x[0]=x,x[1]=t
    a = 688
    b = 40.1
    
    dy_t = dde.grad.jacobian(y, x,i=0,j=1)
    
    k2 = x[:,0:1]/(a+b*x[:,0:1])
    
    eq1 =  k2*y + dy_t
 
    return eq1

def solution1(x,t):
    a = 688
    b = 40.1
    
    y= np.exp(-x*t/(a+b*x))
    
    return y



geom = dde.geometry.Interval(0, 200)
timedomain = dde.geometry.TimeDomain(0, 200)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

def boundary(x, on_boundary):
    return on_boundary and np.isclose(x[0], 0)

def func(x):
    return 1


bc = dde.icbc.DirichletBC(geomtime, func, boundary)

ic = dde.icbc.IC(
    geomtime,
    lambda x: 1,
    lambda _, on_initial: on_initial,
)


def func1(x):
    
    return solution1(x[:,0:1],x[:,1:2])

data = dde.data.TimePDE(geomtime, PDE, [bc,ic] , 3500, 100,train_distribution='pseudo',solution=func1, num_initial=600, num_test=None)

layer_size = [2] + [50] * 3 + [1]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.maps.FNN(layer_size, activation, initializer)

model = dde.Model(data, net)
model.compile("adam", lr=0.001)
losshistory, train_state = model.train(epochs=50000)

X = geomtime.random_points(10000)
err = 1
# itr = 0

# for itr in range(5):
#     if err > 0.005:
new_points = []
for i in range(200):
    f = model.predict(X, operator=PDE)
    err_eq = np.absolute(f)
    err = np.mean(err_eq)
    print("Mean residual: %.3e" % (err))

    x_id = np.argmax(err_eq)
    print("Adding new point:", X[x_id], "\n")
    new_points.append(X[x_id])
    data.add_anchors(X[x_id])
    early_stopping = dde.callbacks.EarlyStopping(min_delta=1e-4, patience=2000)
    model.compile("adam", lr=1e-3)
    model.train(iterations=10000, disregard_previous_best=True, callbacks=[early_stopping])
    model.compile("L-BFGS")
    losshistory, train_state = model.train()
#     itr = itr+1
#     print('itr:',itr)
        
dde.saveplot(losshistory, train_state, issave=True, isplot=True)

# X, y_true = gen_testdata()
# y_pred = model.predict(X)
# print("L2 relative error:", dde.metrics.l2_relative_error(y_true, y_pred))
# np.savetxt("test.dat", np.hstack((X, y_true, y_pred)))

# dde.saveplot(losshistory, train_state, issave=True, isplot=True)

Compiling model...
Building feed-forward neural network...
'build' took 0.064229 s



/Users/zhangtianjie/opt/anaconda3/envs/tf/lib/python3.7/site-packages/deepxde/nn/tensorflow_compat_v1/fnn.py:110: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_constraint=self.kernel_constraint,
2023-09-14 17:04:58.651449: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


'compile' took 0.278239 s

Initializing variables...
Training model...



2023-09-14 17:04:58.875387: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


Step      Train loss                        Test loss                         Test metric
0         [1.13e-04, 9.22e-01, 1.65e+00]    [1.13e-04, 9.22e-01, 1.65e+00]    []  
1000      [1.27e-04, 3.19e-06, 1.80e-05]    [1.27e-04, 3.19e-06, 1.80e-05]    []  
2000      [5.90e-05, 7.63e-07, 3.13e-06]    [5.90e-05, 7.63e-07, 3.13e-06]    []  
3000      [3.61e-05, 5.35e-06, 4.78e-06]    [3.61e-05, 5.35e-06, 4.78e-06]    []  
4000      [2.57e-05, 1.41e-05, 1.52e-05]    [2.57e-05, 1.41e-05, 1.52e-05]    []  
5000      [1.70e-05, 2.37e-07, 2.34e-07]    [1.70e-05, 2.37e-07, 2.34e-07]    []  
6000      [1.56e-05, 3.10e-04, 1.19e-06]    [1.56e-05, 3.10e-04, 1.19e-06]    []  
7000      [1.22e-05, 2.58e-06, 8.85e-08]    [1.22e-05, 2.58e-06, 8.85e-08]    []  
8000      [8.38e-06, 3.25e-08, 5.44e-08]    [8.38e-06, 3.25e-08, 5.44e-08]    []  
9000      [6.22e-06, 2.07e-08, 4.72e-08]    [6.22e-06, 2.07e-08, 4.72e-08]    []  
10000     [4.66e-06, 1.76e-05, 2.38e-06]    [4.66e-06, 1.76e-05, 2.38e-06]    []

...

Mean residual: 4.509e-05
Adding new point: [12.829659   0.1485487] 

Compiling model...
'compile' took 0.339242 s

Training model...

Step      Train loss                        Test loss                         Test metric
83369     [3.88e-09, 3.85e-11, 1.29e-10]    [3.38e-09, 3.85e-11, 1.29e-10]    []  
84000     [3.46e-09, 6.14e-11, 8.27e-11]    [3.22e-09, 6.14e-11, 8.27e-11]    []  
85000     [2.69e-08, 1.13e-07, 5.73e-06]    [2.52e-08, 1.13e-07, 5.73e-06]    []  
Epoch 85370: early stopping

Best model at step 84000:
  train loss: 3.60e-09
  test loss: 3.36e-09
  test metric: []

'train' took 19.928033 s

Compiling model...
'compile' took 0.233571 s

Training model...

Step      Train loss                        Test loss                         Test metric
85370     [3.37e-09, 1.29e-08, 1.49e-08]    [2.89e-09, 1.29e-08, 1.49e-08]    []  
INFO:tensorflow:Optimization terminated with:
  Message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  Objective function value: 0.000000
  

Compiling model...
'compile' took 0.224731 s

Training model...

Step      Train loss                        Test loss                         Test metric
97792     [1.06e-07, 5.86e-09, 9.79e-07]    [1.06e-07, 5.86e-09, 9.79e-07]    []  
INFO:tensorflow:Optimization terminated with:
  Message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  Objective function value: 0.000000
  Number of iterations: 118
  Number of functions evaluations: 149
97941     [2.47e-09, 5.15e-11, 5.91e-11]    [2.19e-09, 5.15e-11, 5.91e-11]    []  

Best model at step 95791:
  train loss: 2.46e-09
  test loss: 2.11e-09
  test metric: []

'train' took 4.039521 s

Mean residual: 3.550e-05
Adding new point: [12.846072   0.4387709] 

Compiling model...
'compile' took 0.277333 s

Training model...

Step      Train loss                        Test loss                         Test metric
97941     [2.54e-09, 5.15e-11, 5.91e-11]    [2.19e-09, 5.15e-11, 5.91e-11]    []  
98000     [4.58e-08, 4.86e-07, 1.47e-07]    [4.


Best model at step 108247:
  train loss: 1.67e-09
  test loss: 1.36e-09
  test metric: []

'train' took 4.737695 s

Mean residual: 2.610e-05
Adding new point: [1.8057843e+02 1.2874027e-01] 

Compiling model...
'compile' took 0.305466 s

Training model...

Step      Train loss                        Test loss                         Test metric
110388    [1.78e-09, 3.20e-11, 4.41e-11]    [1.45e-09, 3.20e-11, 4.41e-11]    []  
111000    [1.60e-09, 3.03e-11, 2.85e-11]    [1.32e-09, 3.03e-11, 2.85e-11]    []  
112000    [2.66e-09, 6.25e-07, 5.83e-07]    [2.27e-09, 6.25e-07, 5.83e-07]    []  
Epoch 112389: early stopping

Best model at step 111000:
  train loss: 1.66e-09
  test loss: 1.37e-09
  test metric: []

'train' took 19.355956 s

Compiling model...
'compile' took 0.247358 s

Training model...

Step      Train loss                        Test loss                         Test metric
112389    [1.64e-09, 5.45e-10, 1.05e-10]    [1.36e-09, 5.45e-10, 1.05e-10]    []  
INFO:tensorflow:Opt

Epoch 124725: early stopping

Best model at step 122724:
  train loss: 1.35e-09
  test loss: 9.43e-10
  test metric: []

'train' took 22.432242 s

Compiling model...
'compile' took 0.419954 s

Training model...

Step      Train loss                        Test loss                         Test metric
124725    [1.28e-09, 1.39e-09, 7.56e-09]    [9.69e-10, 1.39e-09, 7.56e-09]    []  
INFO:tensorflow:Optimization terminated with:
  Message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  Objective function value: 0.000000
  Number of iterations: 5
  Number of functions evaluations: 35
124760    [1.28e-09, 9.95e-12, 9.09e-12]    [9.66e-10, 9.95e-12, 9.09e-12]    []  

Best model at step 124760:
  train loss: 1.30e-09
  test loss: 9.85e-10
  test metric: []

'train' took 5.475214 s

Mean residual: 2.240e-05
Adding new point: [193.83618     0.7657597] 

Compiling model...
'compile' took 0.421979 s

Training model...

Step      Train loss                        Test loss                    

137204    [1.08e-09, 6.88e-11, 3.86e-11]    [7.48e-10, 6.88e-11, 3.86e-11]    []  

Best model at step 137204:
  train loss: 1.19e-09
  test loss: 8.55e-10
  test metric: []

'train' took 7.513038 s

Mean residual: 1.938e-05
Adding new point: [2.554278  1.8622324] 

Compiling model...
'compile' took 0.446409 s

Training model...

Step      Train loss                        Test loss                         Test metric
137204    [1.12e-09, 6.88e-11, 3.86e-11]    [7.48e-10, 6.88e-11, 3.86e-11]    []  
138000    [1.27e-09, 5.43e-12, 3.45e-11]    [7.55e-10, 5.43e-12, 3.45e-11]    []  
139000    [9.18e-10, 8.97e-11, 2.27e-10]    [6.68e-10, 8.97e-11, 2.27e-10]    []  
Epoch 139205: early stopping

Best model at step 137204:
  train loss: 1.23e-09
  test loss: 8.55e-10
  test metric: []

'train' took 24.206310 s

Compiling model...
'compile' took 0.345333 s

Training model...

Step      Train loss                        Test loss                         Test metric
139205    [2.05e-08, 1.36e-

In [ ]:
import pandas as pd
test1=np.loadtxt('test.dat', unpack = True)
df = pd.DataFrame(test1.T, columns = ['x','t','exact','predict'])
df

In [ ]:
import pandas as pd
train1=np.loadtxt('train.dat', unpack = True)
train1 = pd.DataFrame(train1.T)
test1=np.loadtxt('test.dat', unpack = True)
df = pd.DataFrame(test1.T, columns = ['x','y','u','s'])
x = df['x']
y = df['y']
ext = df['u']
pre = df['s']
from scipy.interpolate import griddata
N = 1000; 
#npts = 2000*2000

# grid_x,grid_y = np.meshgrid(np.linspace(x.min(),x.max(),N),np.linspace(y.min(),y.max(),N))
grid_x,grid_y = np.meshgrid(np.linspace(0,200,N),np.linspace(0,200,N))
# grid_x1,grid_y1 = np.meshgrid(np.linspace(0,1,500),np.linspace(0,1,500))

#px, py = np.random.choice(x, npts), np.random.choice(y, npts)
grid_ext = griddata((x,y) ,ext, (grid_x, grid_y), method='linear')
grid_pre = griddata((x,y) ,pre, (grid_x, grid_y), method='linear')

grid_err = grid_pre-grid_ext

rng = np.random.default_rng()
# points = rng.random((1000, 2))

# a=np.zeros(1000)
# a[:]=np.nan
# grid_z2 = griddata(points ,  a , (grid_x1, grid_y1), method='cubic')
#grid_z1[999:2000,999:2000]=0
#scipy.interpolate.griddata(X,Y,X**2+Y**2,XI,YI)
plt.contourf(grid_x,grid_y, grid_pre,cmap="viridis",origin='lower', aspect='auto',interpolation='nearest')
plt.scatter(train1[0],train1[1],c='black',s=1)
# plt.scatter(test1[0],test1[1],c='yellow',s=1)
# plt.axis('equal')
plt.tight_layout()
# plt.axis('off')
plt.xlabel('time')
plt.xlim(0,200)
plt.ylabel('shear rate')


In [ ]:
fig, ax = plt.subplots()
h=ax.contourf(grid_x,grid_y, grid_ext,cmap="viridis",origin='lower', aspect='auto',interpolation='nearest')
plt.xlabel('Time')
plt.ylabel('Shear Rate')

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(h, cax=cax)
plt.savefig('solution.png',dpi=600,bbox_inches = 'tight')

In [ ]:
fig, ax = plt.subplots()
h=ax.contourf(grid_x,grid_y, grid_pre,cmap="viridis",origin='lower', aspect='auto',interpolation='nearest')
plt.xlabel('Time')
plt.ylabel('Shear Rate')

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(h, cax=cax)
plt.savefig('prediction.png',dpi=600,bbox_inches = 'tight')

In [ ]:
fig, ax = plt.subplots()
h=ax.contourf(grid_x,grid_y, grid_err,cmap="viridis",origin='lower', aspect='auto',interpolation='nearest')
plt.xlabel('Time')
plt.ylabel('Shear Rate')

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(h, cax=cax)
plt.savefig('error.png',dpi=600,bbox_inches = 'tight')

# Test

In [ ]:
x = np.linspace(0, 200, 200)
t = np.linspace(0, 200, 200)
X, T = np.meshgrid(x, t)

X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
prediction = model.predict(X_star, operator=None)

pre = griddata(X_star, prediction[:, 0], (X, T), method="cubic")

fig, ax = plt.subplots()
h=ax.contourf(X,T, pre,cmap="viridis",origin='lower', aspect='auto',interpolation='nearest')
plt.xlabel('$t(s)$',fontproperties = 'Times New Roman', size = 14)
plt.ylabel('$\dot{\gamma } ( s^{-1}) $ ',fontproperties = 'Times New Roman', size = 14 )
plt.title('$\mathbf{\lambda _{RheologyNet}}$ ',fontproperties = 'Times New Roman', size = 14)


from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(h, cax=cax)
plt.savefig('pinn_of_λ.png',dpi=600,bbox_inches = 'tight')

# exact solution

In [ ]:
train1=np.loadtxt('train.dat', unpack = True)
train1 = pd.DataFrame(train1.T)
ext = griddata(X_star, solution1(X_star[:,0],X_star[:,1]), (X, T), method="cubic")
fig, ax = plt.subplots()
h=ax.contourf(X,T, ext,cmap="viridis",origin='lower', aspect='auto',interpolation='nearest')
plt.xlabel('$t(s)$',fontproperties = 'Times New Roman', size = 14)
plt.ylabel('$\dot{\gamma } ( s^{-1}) $ ',fontproperties = 'Times New Roman', size = 14 )
plt.title('$\mathbf{\lambda _{FEA}}$ ',fontproperties = 'Times New Roman', size = 14)
plt.scatter(train1[0],train1[1],c='black',s=1)


from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(h, cax=cax)
plt.savefig('solution_of_λ.png',dpi=600,bbox_inches = 'tight')



In [ ]:
err=abs(pre-ext)
fig, ax = plt.subplots()
h=ax.contourf(X,T, err,cmap="viridis",origin='lower', aspect='auto',interpolation='nearest')
plt.xlabel('$t(s)$',fontproperties = 'Times New Roman', size = 14)
plt.ylabel('$\dot{\gamma } ( s^{-1}) $ ',fontproperties = 'Times New Roman', size = 14 )

plt.title(r'$\mathbf{\varepsilon_{\lambda ,RheologyNet} }$ ',fontproperties = 'Times New Roman', size = 14 )

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(h, cax=cax)
plt.savefig('pinn_err_λ.png',dpi=600,bbox_inches = 'tight')

In [ ]:
train=np.loadtxt('train.dat', unpack = True)
df = pd.DataFrame(train.T)
df

In [ ]:
import pandas as pd
train=np.loadtxt('train.dat', unpack = True)
# train= pd.DataFrame(train.T)
# test1=np.loadtxt('test.dat', unpack = True)
df = pd.DataFrame(train.T, columns = ['x','y','u'])
x = df['x']
y = df['y']
z = df['u']
from scipy.interpolate import griddata
N = 1000; 
#npts = 2000*2000

grid_x,grid_y = np.meshgrid(np.linspace(x.min(),x.max(),N),np.linspace(y.min(),y.max(),N))
# grid_x1,grid_y1 = np.meshgrid(np.linspace(0,1,500),np.linspace(0,1,500))

#px, py = np.random.choice(x, npts), np.random.choice(y, npts)
grid_z1 = griddata((x,y) ,z, (grid_x, grid_y), method='linear')

rng = np.random.default_rng()
# points = rng.random((1000, 2))

# a=np.zeros(1000)
# a[:]=np.nan
# grid_z2 = griddata(points ,  a , (grid_x1, grid_y1), method='cubic')
#grid_z1[999:2000,999:2000]=0
#scipy.interpolate.griddata(X,Y,X**2+Y**2,XI,YI)
plt.contourf(grid_x,grid_y, grid_z1,cmap="viridis",origin='lower', aspect='auto',interpolation='nearest')
# plt.scatter(train1[0],train1[1],c='red',s=1)
# plt.scatter(test1[0],test1[1],c='black',s=1)
# plt.axis('equal')
plt.tight_layout()
# plt.axis('off')
plt.xlabel('time')
plt.xlim(0,200)
plt.ylabel('shear rate')


In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error

train=np.loadtxt('train.dat', unpack = True)
x_train = train[[0,1]].T
y_train = train[2]

clf = MLPRegressor(hidden_layer_sizes=(50,50,50),max_iter=50000)

clf.fit(x_train, y_train)


In [ ]:
x = np.linspace(0, 200, 200)
t = np.linspace(0, 200, 200)
X, T = np.meshgrid(x, t)

X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))

prediction = clf.predict(X_star)

pre_nn = griddata(X_star, prediction, (X, T), method="cubic")

fig, ax = plt.subplots()
h=ax.contourf(X,T, pre_nn,cmap="viridis",origin='lower', aspect='auto',interpolation='nearest')
plt.xlabel('$t(s)$',fontproperties = 'Times New Roman', size = 14)
plt.ylabel('$\dot{\gamma } ( s^{-1}) $ ',fontproperties = 'Times New Roman', size = 14 )
plt.title('$\mathbf{\lambda _{FNN}}$ ',fontproperties = 'Times New Roman', size = 14)

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(h, cax=cax)
plt.savefig('FNN_pre.png',dpi=600,bbox_inches = 'tight')

In [ ]:
err_nn = abs(pre_nn-ext)
fig, ax = plt.subplots()
h=ax.contourf(X,T, err_nn,cmap="viridis",origin='lower', aspect='auto',interpolation='nearest')
plt.xlabel('$t(s)$',fontproperties = 'Times New Roman', size = 14)
plt.ylabel('$\dot{\gamma } ( s^{-1}) $ ',fontproperties = 'Times New Roman', size = 14 )
plt.title(r'$\mathbf{\varepsilon_{\lambda ,FNN} }$ ',fontproperties = 'Times New Roman', size = 14)

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(h, cax=cax)
plt.savefig('FNN_err.png',dpi=600,bbox_inches = 'tight')

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(pre,ext)

In [ ]:
mean_squared_error(pre_nn,ext)

In [ ]:
d={'RheologyNet of lambda':[0.0003,0.0021,0.0005],'FNN of lambda':[0.0023,0.0034,0.0041],'RheologyNet of shear':[0.0003,0.0021,0.0005],'FNN of shear':[0.0023,0.0034,0.0041]}
data=pd.DataFrame(data=d)
plt.boxplot(data)
plt.xticks([1,2,3,4],['RheologyNet','FNN','Rhe','N'])
# plt.yscale('log')
# plt.ylim(0,1)
